- 1.使用 Xavier 初始化。
- 2.每次只取一个batch，减少内存占用
- 3.字符embedding

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from gensim.models import KeyedVectors
import pickle
import os
import sys
import shutil
import time


time0 = time.time()
print('Starting ...')
model_name = 'ch3-1-cnn-256-2345'                    # 模型名称
W_embedding = np.load('/home1/huangyongye/mygithub/zhihu/data/ch_W_embedding.npy').astype(np.float32)            # 导入预训练好的词向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置
local_scores_path = '../local_scores/' + model_name + '.npy'


if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    print('removed the existing summary files.')
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建
    
# ##################### config ######################
n_step1 = max_len1 = 52                   # title句子长度
n_step2 = max_len2 = 300                   # content 长度
input_size = embedding_size = 256       # 字向量长度
n_class = 1999                          # 类别总数
filter_sizes = [2,3,4,5]                  # 卷积核大小
n_filter = 256                          # 每种卷积核的个数
fc_hidden_size = 1024                   # fc 层节点数
n_filter_total = n_filter * len(filter_sizes)
valid_num = 100000
seed_num = 13
tr_batch_size = 128
te_batch_size = 128
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
removed the existing summary files.
Prepared, costed time 0.0118241 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
双端 GRU，知乎问题多标签分类。
'''
print('Building model ...')
keep_prob = tf.placeholder(tf.float32, [])
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()                       # BN 层中所有的更新操作


def weight_variable(shape, name, initializer=None):
    """Create a weight variable with appropriate initialization."""
    if initializer is None:
        initializer = tf.contrib.layers.xavier_initializer()
    return tf.get_variable(name=name, shape=shape, initializer=initializer, dtype=tf.float32)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.get_variable(name=name, initializer=initial, dtype=tf.float32)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages


with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, n_step1], name='X1_input')
    X2_inputs = tf.placeholder(tf.int64, [None, n_step2], name='X2_input')
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')    

with tf.device('/gpu:0'):
    with tf.variable_scope('embedding') as vs:
        embedding = tf.get_variable(name="W_embedding", shape=W_embedding.shape, 
                            initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune

def textcnn(X_inputs, n_step):
    """build the TextCNN network. Return the h_drop"""
    # X_inputs.shape = [batchsize, n_step]  ->  inputs.shape = [batchsize, n_step, embedding_size]
    inputs = tf.nn.embedding_lookup(embedding, X_inputs)  
    inputs = tf.expand_dims(inputs, -1)
    pooled_outputs = list()
    for i, filter_size in enumerate(filter_sizes):
        with tf.variable_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [filter_size, embedding_size, 1, n_filter]
            W_filter = weight_variable(shape=filter_shape, name='W_filter')
            beta = bias_variable(shape=[n_filter], name='beta_filter')
            tf.summary.histogram('beta_filter', beta)
            conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
            conv_bn, update_ema = batchnorm(conv, tst, n_updates, beta, convolutional=True)    # 在激活层前面加 BN
            # Apply nonlinearity, batch norm scaling is not useful with relus
            # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
            h = tf.nn.relu(conv_bn, name="filter_relu")
            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(h,ksize=[1, n_step - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],padding='VALID',name="pool")
            pooled_outputs.append(pooled)
            update_emas.append(update_ema)
    h_pool = tf.concat(pooled_outputs, 3)
    h_pool_flat = tf.reshape(h_pool, [-1, n_filter_total]) 
    return h_pool_flat    # shape = [-1, n_filter_total]
    
    
with tf.variable_scope('cnn-title'):
    output_title = textcnn(X1_inputs, n_step1)
with tf.variable_scope('cnn-content'):
    output_content = textcnn(X2_inputs, n_step2)
with tf.variable_scope('fc-bn-layer'):
    output = tf.concat([output_title, output_content], axis=1)
    W_fc = weight_variable([n_filter_total*2, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = bias_variable([fc_hidden_size], name="beta_fc")
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob, name="fc_dropout")

with tf.variable_scope('out_layer'):
    W_out = weight_variable(shape=[fc_hidden_size, n_class], name='Weight_out')
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  #每个类别的分数 scores
    
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)

#  ------------- 优化器设置 ---------------------
global_step = tf.Variable(0, trainable=False, name='Global_Step')
update_global_step = tf.assign(global_step, global_step+1)
starter_learning_rate = 5e-4
decay_step = 10000
# decay_step = 25  # 测试用
decay = 0.92 
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_step, decay, staircase=False)
with tf.variable_scope('AdamOptimizer1'):
    tvars1 = tf.trainable_variables()
    grads1 = tf.gradients(cost, tvars1)
    optimizer1 = tf.train.AdamOptimizer(learning_rate)
    train_op1 = optimizer1.apply_gradients(zip(grads1, tvars1),
        global_step=global_step)

update_op = tf.group(*update_emas)   # 更新 BN 参数    

# summary
merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the TextCNN model.'

Building model ...
Finished creating the TextCNN model.


## 导入数据

In [3]:
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


data_train_path = '/home2/huangyongye/zhihu/ch-data/data_train/'
data_valid_path = '/home2/huangyongye/zhihu/ch-data/data_valid/'
tr_batches = os.listdir(data_train_path)   # batch 文件名列表
va_batches = os.listdir(data_valid_path)
n_tr_batches = len(tr_batches)
n_va_batches = len(va_batches)

# 测试用
# n_tr_batches = 2000  
# n_va_batches = 100

print('n_tr_batch=%d' % n_tr_batches)
print('n_va_batches=%d' % n_va_batches)


def get_batch(data_path, batch_id, title_len=n_step1):
    """get a batch from data_path"""
    new_batch = np.load(data_path + str(batch_id) + '.npz')
    X_batch = new_batch['X']
    y_batch = new_batch['y']
    X1_batch = X_batch[:, :title_len]
    X2_batch = X_batch[:, title_len:]
    return [X1_batch, X2_batch, y_batch]


def valid_epoch(data_path=data_valid_path):
    """Test on the valid data."""
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    _global_step = sess.run(global_step)
    for i in xrange(n_va_batches):
        [X1_batch, X2_batch, y_batch] = get_batch(data_path, i)
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, y_pred]  
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:_global_step}
        summary, _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / n_va_batches
    return mean_cost, precision, recall, f1

print('Every thing prepared!')

n_tr_batch=22656
n_va_batches=782
Every thing prepared!


##  模型训练

之前的训练速度 9334.15 s/epoch

In [4]:
# 测试
# valid_step = 1000        # 每 valid_step 就进行一次 valid 运算
# max_max_epoch = 2       # 最多迭代的次数

# 正式
valid_step = 10000        # 每 valid_step 就进行一次 valid 运算
max_max_epoch = 7       # 最多迭代的次数

print('tr_batch_num=%d' % n_tr_batches)
saver = tf.train.Saver(max_to_keep=5)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.39
model_num = 0

tr_batch_num=22656


In [5]:
time0 = time.time()
train_op = train_op1 

for epoch in xrange(max_max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    print 'EPOCH %d, lr= %g' % (epoch+1, sess.run(learning_rate))    
    for batch in xrange(n_tr_batches): 
        _global_step = sess.run(global_step)
        if (_global_step+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%g s' % (
                    _global_step, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('the save path is ', save_path) 
                
        batch_id = batch_indexs[batch]
        [X1_batch, X2_batch, y_batch] = get_batch(data_train_path, batch_id, n_step1)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, train_op, update_op]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, batch_size:_batch_size, 
                     keep_prob:0.5, tst:False, n_updates:_global_step}
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if _global_step % 100:
            train_writer.add_summary(summary, _global_step)
            batch_id = np.random.randint(0, n_va_batches)   # 随机选一个验证batch
            [X1_batch, X2_batch, y_batch] = get_batch(data_valid_path, batch_id, n_step1)
            y_batch = to_categorical(y_batch)
            _batch_size = len(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch,
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:_global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, _global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    _global_step, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > 0.001):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, lr= 0.0005
Global_step=9999: valid cost=0.00869695; p=1.0698, r=0.446747, f1=0.315143, time=4630.35 s
Global_step=19999: valid cost=0.00737875; p=1.14491, r=0.476356, f1=0.336394, time=3989.64 s
EPOCH 2, lr= 0.000413931
Global_step=29999: valid cost=0.00695052; p=1.1958, r=0.496648, f1=0.350907, time=3911.53 s
Global_step=39999: valid cost=0.00653881; p=1.21148, r=0.501467, f1=0.354662, time=3899.04 s
EPOCH 3, lr= 0.000342677
Global_step=49999: valid cost=0.00626661; p=1.25759, r=0.518539, f1=0.367152, time=3901.55 s
Global_step=59999: valid cost=0.00576899; p=1.27733, r=0.525855, f1=0.372502, time=3920.19 s
EPOCH 4, lr= 0.000283689
Global_step=69999: valid cost=0.00552376; p=1.29649, r=0.533324, f1=0.377879, time=3957.34 s
Global_step=79999: valid cost=0.00555276; p=1.3109, r=0.53831, f1=0.381606, time=3951.29 s
Global_step=89999: valid cost=0.00576692; p=1.30804, r=0.537681, f1=0.381048, time=4109.23 s
EPOCH 5, lr= 0.000234856
Global_step=99999: valid cost=0.00513846; p=1.33

In [6]:
sess.run(learning_rate)

0.00013325211

继续迭代

## 本地测试
使用 seed13 的前 10万条数据.

In [18]:
a = np.load('../ch-data/data_test/1.npy')

In [19]:
a.shape

(128, 352)

In [8]:
# 保存 本地测试的标注数据
local_scores_path = '../local_scores/' + model_name + '.npy'

In [17]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

from tqdm import tqdm

# 导入测试数据
def local_predict(scores_path=local_scores_path, data_path=data_valid_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    predict_scores = list()
    _global_step = sess.run(global_step)
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, y_batch] = get_batch(data_path, i)
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:_global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

  0%|          | 2/782 [00:00<01:06, 11.75it/s]

local predicting ...


100%|██████████| 782/782 [01:09<00:00, 11.26it/s]


local valid p=1.36686, r=0.559213, f1=0.396852; speed=69.9659 s/epoch
Writed the scores into ../local_scores/ch3-1-cnn-256-2345.npy, time 70.577 s


#### 多个 epoch 

In [ ]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

from tqdm import tqdm

marked_labels_list = np.load('../data/marked_labels_list.npy')
# # 求 softmax
def _softmax(score):
    """对一个样本的输出类别概率进行 softmax 归一化.
    score: arr.shape=[1999].
    """
    max_sc = np.max(score)   # 最大分数
    score = score - max_sc
    exp_sc = np.exp(score)
    sum_exp_sc = np.sum(exp_sc)
    softmax_sc = exp_sc / sum_exp_sc
    return softmax_sc    # 归一化的结果
    
def softmax(scores):
    """对所有样本的输出概率进行 softmax 归一化处理。
    scores: arr.shape=[n_sample, 1999].
    """
    softmax_scs = map(_softmax, scores)
    return np.asarray(softmax_scs)


# 导入测试数据
def local_predict(epoch, scores_path=local_scores_path, data_path=data_valid_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    saver = tf.train.Saver()
    best_model_path = model_path + '-' + str(epoch)  # 导入最优模型
    saver.restore(sess, best_model_path)
    print('Finished loading model.')

    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    predict_scores = list()
    _global_step = sess.run(global_step)
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, y_batch] = get_batch(data_path, i)
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,  y_inputs:y_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:_global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.vstack(np.asarray(predict_scores))
    predict_scores = softmax(predict_scores)
    return predict_scores
    
local_scores_path = '../local_scores/' + model_name + '-all.npy'
sum_scores = np.zeros((len(marked_labels_list), 1999), dtype=float)   
last_f1 = 0.396852
valid_epoch = [2,3,4]
for epoch in valid_epoch:
    score =  np.vstack(local_predict(epoch))
    sum_scores = sum_scores + score
predict_labels_list = map(lambda label: label.argsort()[-1:-6:-1], sum_scores) # 取最大的5个下标
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print('p=%g\tr=%g\tf1=%g;' % ( precision, recall, f1))
if f1 > last_f1:
    np.save(local_scores_path, sum_scores)
    print('save scores to %s' % local_scores_path)


## 对测试数据进行预测

In [22]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

# 导入测试数据
def predict():
    """预测测试集结果，并保存到  result.csv 文件中。"""
    data_path = '../ch-data/data_test/'
    n_te_batches = len(os.listdir(data_path))
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    _global_step = sess.run(global_step)
    for i in tqdm(xrange(n_te_batches)):
        X_batch = np.load(data_path + str(i) + '.npy')
        X1_batch = X_batch[:, :n_step1]
        X2_batch = X_batch[:, n_step1:]
        _batch_size = len(X_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:_global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return predict_labels_list, np.vstack(predict_scores)

def write_result(predict_labels_list, result_path):
    """把结果写到 result.csv 中"""
    eval_question = np.load('../data/eval_question.npy')
    with open('../data/sr_topic2id.pkl', 'rb') as inp:
        sr_topic2id = pickle.load(inp)
        sr_id2topic = pickle.load(inp)
    pred_labels = np.asarray(predict_labels_list).reshape([-1])
    pred_topics = sr_id2topic[pred_labels].values.reshape([-1, 5])   # 转为 topic
    df_result = pd.DataFrame({'question':eval_question, 'tid0': pred_topics[:,0], 'tid1':pred_topics[:, 1],
                         'tid2': pred_topics[:,2], 'tid3':pred_topics[:,3],'tid4': pred_topics[:,4]})
    df_result.to_csv(result_path, index=False, header=False)
    print('Finished writing the result')
    return df_result

In [23]:
%time predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print('len(df_result)=',len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print('predict_scores.shape=',predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished saving the result!')

100%|██████████| 1699/1699 [02:10<00:00, 13.03it/s]


CPU times: user 2min 1s, sys: 12.6 s, total: 2min 14s
Wall time: 2min 10s
Finished writing the result
('len(df_result)=', 217360)
Saving the predict_scores into ../scores/ch3-1-cnn-256-2345.npy
('predict_scores.shape=', (217360, 1999))
Finished saving the result!


In [16]:
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,-7506384235581390893,4610596224687453206,2919247920214845195,-5932391056759866388,-6306904715218704629
1,6649324930261961840,-240041917918953337,2858911571784840089,3418451812342379591,2382911985227044227,3383016985780045156
2,-4251899610700378615,2919247920214845195,-5265476641576484497,-3315241959305847628,-7358589937244777363,-429636223750539488
3,6213817087034420233,-8655945395761165989,-4966205278807386328,5804619920623030604,7476760589625268543,244937959911721367
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,7951349602759061249,1870872991887862017


In [25]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,2919247920214845195,-6839713564940654454,-7506384235581390893,-5932391056759866388
1,6649324930261961840,3418451812342379591,2858911571784840089,3383016985780045156,2382911985227044227,-240041917918953337
2,-4251899610700378615,2919247920214845195,-5265476641576484497,-4200589112729926854,-5932391056759866388,-8955755936847898358
3,6213817087034420233,5804619920623030604,-8655945395761165989,-4966205278807386328,7476760589625268543,6913268882499481459
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,6018641953300645757,7951349602759061249


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
